In [1]:
import time
import os
import paramiko
from io import StringIO
import time
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from IPython.display import clear_output
import datetime

In [2]:
# SSH credentials and remote CSV file path
host = 'lxslc7.ihep.ac.cn'
username = os.environ.get('IHEP_USERNAME')
password = os.environ.get('IHEP_PASSWORD')
remote_csv_file = '/hpcfs/cepc/higgs/frank/GNN/bepc.magnet.movingE/output/summaries_0.csv'
# remote_csv_file = '/hpcfs/cepc/higgs/frank/GNN/quick_test/output/summaries_0.csv'

# Establish an SSH connection
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, username=username, password=password)
print(f'Successfully connected to {username}@{host}')

Successfully connected to frank@lxslc7.ihep.ac.cn


In [3]:
def update_csv():
    sftp = ssh.open_sftp()
    with sftp.open(remote_csv_file, 'r') as file:
        remote_csv_data = file.read().decode('utf-8')
        timestamp = sftp.stat(remote_csv_file).st_mtime

        dt_object = datetime.datetime.fromtimestamp(timestamp)

        # Parse and display the CSV data
        df = pd.read_csv(StringIO(remote_csv_data))
        return df, f"{dt_object.year} - {dt_object.month} - {dt_object.day}   {dt_object.hour} : {dt_object.minute}"


def update_fig(df, t):
    fig = go.FigureWidget()

    y_max, y_min = df.max()[['train_loss', 'valid_loss']].max(), df.min()[['train_loss', 'valid_loss']].min()

    df_new = df[['epoch', 'train_loss', 'valid_loss', 'valid_dp_mean', 'valid_dp_std']]. \
        groupby('epoch').transform("mean").drop_duplicates(keep='last', subset=['train_loss'])

    mu_max = df.abs().max()['valid_dp_mean']
    std_max = df.abs().max()['valid_dp_std']

    fig.data = []
    fig.add_trace(
        go.Scatter(
            x=df_new.index,
            y=df_new['valid_dp_mean'],
            mode='lines', name="itr: p mean",
            line=dict(dash='dot', color="#ff7f0e"), yaxis='y3',
            legendgroup="Momentum",
            legendgrouptitle_text="Momentum",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df_new.index,
            y=df_new['valid_dp_std'],
            mode='lines', name="itr: p std",
            line=dict(dash='dot', color="#64a05f"), yaxis='y4',
            legendgroup="Momentum",
        )
    )

    fig.add_trace(
        go.Scatter(
            x=df.index, y=df['train_loss'], mode='lines', name="itr: train", legendgroup="Itr Loss",
            legendgrouptitle_text="Itr Loss", line=dict(dash='dot'))
    )
    fig.add_trace(
        go.Scatter(
            x=df.index, y=df['valid_loss'], mode='lines', name="itr: valid", legendgroup="Itr Loss",
            line=dict(dash='dot')
        ))
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=-np.log10(1 - df['valid_acc']),
            # y=df['valid_acc'],
            mode='lines', name="itr: accuracy",
            line=dict(dash='dot', color="#11ADF0"), yaxis='y2',
            legendgroup="Edge Label",
            legendgrouptitle_text="Edge Label",
        )
    )

    fig.add_trace(go.Scatter(
        x=df_new.index, y=df_new['train_loss'], mode='lines+markers', name="epoch: train",
        legendgroup="Epoch Loss", legendgrouptitle_text="Epoch Loss"
    ))
    fig.add_trace(go.Scatter(
        x=df_new.index, y=df_new['valid_loss'], mode='lines+markers', name="epoch: valid",
        legendgroup="Epoch Loss"
    ))

    for lr in df.drop_duplicates(keep="first", subset=['lr'])['lr'].items():
        print(lr)
        fig.add_vline(x=lr[0], line_width=2, line_dash="dash", line_color="grey")
        fig.add_annotation(
            text=f'$\eta = {lr[1]}$',
            x=lr[0] + 0.5, y=np.log10(y_min) * 1.02,  # Set the position using numeric values
            xanchor='left',
            # yanchor='bottom',
            showarrow=False,
            font=dict(size=14, color='grey')
        )

    fig.update_layout(
        title=f"{t} --> Epoch: {df['epoch'].iloc[-1]}, Iteration: {df['itr'].iloc[-1]}",
        width=1200,
        height=700,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis=dict(
            title_text="Total iteration",
            showgrid=False,
            mirror=True,
            showline=True,
            zeroline=False,
            linewidth=2,
            linecolor='#666666', gridcolor='#d9d9d9',
            domain=[0.1, 0.85],
        ),
        yaxis=dict(
            title=r"Loss",
            titlefont=dict(color="#d62728"),
            tickfont=dict(color="#d62728"),
            showgrid=False,
            linecolor="#d62728", gridcolor='#d9d9d9',
            zeroline=False,
            type="log",
            range=[np.log10(y_min) * 1.05, np.log10(y_max) * 1.05],
        ),
        yaxis2=dict(
            # title=r"$\textrm{95% CI }{\Large \kappa_{2V}}\textrm{ Interval}$",
            title=r"$-log(\epsilon_{error})$",
            titlefont=dict(color="#11ADF0"),
            tickfont=dict(color="#11ADF0"),
            anchor="free",
            overlaying="y",
            side="left",
            position=0.0,
            showgrid=False,
            linecolor="#11ADF0", gridcolor='#d9d9d9',
            zeroline=False,
            # type="log",
            # range=[3.0, 3.7],
        ),
        yaxis3=dict(
            title=r"$\mu^{p}_{\textrm{valid}}$",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            anchor="x",
            overlaying="y",
            side="right",
            showgrid=False,
            linecolor="#ff7f0e", gridcolor='#d9d9d9',
            zeroline=False,
            range=[-mu_max * 1.05, mu_max * 1.05]
        ),
        yaxis4=dict(
            title=r"$\sigma^{p}_{\textrm{valid}}$",
            titlefont=dict(color="#64a05f"),
            tickfont=dict(color="#64a05f"),
            anchor="free",
            overlaying="y",
            side="right",
            position=0.95,
            showgrid=False,
            linecolor="#64a05f", gridcolor='#d9d9d9',
            zeroline=False,
            range=[0.0, std_max],
        ),
    )

    return fig

In [6]:
df, t = update_csv()
fig = update_fig(df, t)
fig.show()
while True:
    df, t = update_csv()
    fig = update_fig(df, t)
    time.sleep(30)
    clear_output(wait=False)
    fig.show()

(0, 0.0001)
(16, 0.0001375)
(32, 0.000175)
(48, 0.0002125)
(64, 0.00025)
(80, 0.0002875)
(96, 0.000325)
(112, 0.0003625)
(128, 0.0004)
(1040, 4e-05)
(1360, 4.000000000000001e-06)


(0, 0.0001)
(16, 0.0001375)
(32, 0.000175)
(48, 0.0002125)
(64, 0.00025)
(80, 0.0002875)
(96, 0.000325)
(112, 0.0003625)
(128, 0.0004)
(1040, 4e-05)
(1360, 4.000000000000001e-06)


KeyboardInterrupt: 

In [9]:
df

,lr,train_loss,l1,l2,grad_norm,train_batches,itr,epoch,valid_loss,valid_acc,valid_TP,valid_FP,valid_TN,valid_FN,valid_batches,valid_sum_total,valid_dp_mean,valid_dp_std
0,0.000100,1.660715,3854.724130,27.854984,34.920914,18,0,0,1.340133,0.695645,33292,914349,2226913,74522,8,3249076,-inf,NaN
1,0.000100,1.268027,3846.899183,27.834845,8.550172,18,1,0,1.227834,0.447402,76091,1771271,1383535,31548,8,3262445,NaN,NaN
2,0.000100,1.216197,3839.218227,27.815647,1.823713,18,2,0,1.195077,0.420860,89387,1852989,1270857,18829,8,3232062,NaN,NaN
3,0.000100,1.186391,3831.836115,27.797698,1.104583,18,3,0,1.174627,0.459808,88880,1750467,1417270,18991,8,3275608,NaN,NaN
4,0.000100,1.167692,3824.832981,27.780975,1.426662,18,4,0,1.158819,0.424654,94041,1791329,1238341,13862,8,3137573,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,0.000004,0.002177,3784.428899,28.223597,0.052311,18,11,99,0.002166,0.996360,108052,11698,3151936,211,8,3271897,NaN,NaN
1596,0.000004,0.002216,3784.198110,28.222784,0.122287,18,12,99,0.002255,0.996376,107920,11826,3210745,244,8,3330735,NaN,NaN
1597,0.000004,0.002177,3783.962580,28.221958,0.050090,18,13,99,0.002178,0.996498,108086,11271,3166994,240,8,3286591,NaN,NaN
1598,0.000004,0.002162,3783.728585,28.221148,0.046907,18,14,99,0.002127,0.996510,107883,11046,3112477,233,8,3231639,NaN,NaN
